# Predict Workout

Predict Workout Day and distance

In [ ]:
#!pip install sklearn

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

filepath = "data/"

def loadCleanData():
    df = pd.read_csv(filepath+ 'activities_clean.csv')
    df['Week'] = pd.to_datetime(df.Week)
    df['Activity Date'] = pd.to_datetime(df['Activity Date']) + timedelta(hours=3)
    return df

data = loadCleanData()
data.head()

### Feature Extraction

In [ ]:
data['year'] = list(map(lambda t : t.year, data['Activity Date']))

In [ ]:
data['month'] = list(map(lambda t : t.month, data['Activity Date']))

In [ ]:
data['day'] = list(map(lambda t : t.day, data['Activity Date']))

In [ ]:
data['hour'] = list(map(lambda t : t.hour, data['Activity Date']))

In [ ]:
data['minute'] = list(map(lambda t : t.minute, data['Activity Date']))

In [ ]:
data['dayOfWeek'] = list(map(lambda t : t.dayofweek, data['Activity Date']))

In [ ]:
data['isWeekend'] = ((pd.DatetimeIndex(data['Activity Date']).dayofweek) // 5 == 1).astype(int)

#### Weather Data Export

In [ ]:
import requests
api_key = "API_KEY"
city = "Istanbul" 
date = "2020-12-25"
hour = 8

def get_weather(api_key,city,date,hour):
    endpoint = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&q={city}&format=json&date={date}"
    response = requests.get(endpoint)
    api_results=response.json()

    # Weather data is splitted into 3hour slices, in below code finding hour slot for weather
    hour_slice = int(hour/3) 

    # Selecting weather hour slice 
    temp = api_results['data']['weather'][0]['hourly'][hour_slice]

    # Return weather details
    temp_c = temp['tempC']
    windspeed_miles = temp['windspeedMiles']
    weather_desc = temp['weatherDesc'][0]['value']

    return temp_c, windspeed_miles, weather_desc

temp_data = get_weather(api_key,city,date,hour)
temp_data


#### Get Weather Data and Save 

In [ ]:
## iterate for all ride date to extract weather
#for index, row in data.iterrows():
#    temp, wind, weatherDesc = get_weather(api_key,city,row['Activity Date'],row['hour'])
#    data['temp'][index] = temp
#    data['wind'][index] = wind 
#    data['weatherDesc'][index] = weatherDesc
#    print(row['Activity Date'],temp, wind, weatherDesc)

In [ ]:
## Export corolated data
#data.to_csv('data/activities_corolated.csv', index=False)
#print('Successfully saved!')

### Transform Data

In [ ]:
# convert string data to numeric value for decision making
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(data['Activity Type'])
le.classes_

In [ ]:
vals = le.transform(le.classes_)
vals

In [ ]:
data['rideType'] = le.transform(data['Activity Type'])
data['rideType']

### Prepare features

In [ ]:
# Select features as list of array
X = data[['month','day','hour','minute','dayOfWeek','isWeekend','temp','wind']]
X = X.to_numpy()
X

### Prepare Predict

In [ ]:
Y_distance = data['Distance']
Y_distance = Y_distance.to_numpy()
Y_distance

In [ ]:
Y_rideType = data['rideType']
Y_rideType = Y_rideType.to_numpy()
Y_rideType

In [ ]:
Y = data[['Distance','rideType']]
Y = Y.to_numpy()
Y

## Feature Evaluation

In [ ]:
# ride distribution per hour
data.plot(kind='scatter', x='hour', y='Distance', xticks=data['hour'], figsize=(14,8))

In [ ]:
# ride distribution per hour
data.plot(kind='scatter', x='dayOfWeek', y='Distance', xticks=data['dayOfWeek'], figsize=(14,8))

In [ ]:
# ride distribution per hour
data.plot(kind='scatter', x='isWeekend', y='Distance', xticks=data['isWeekend'], figsize=(14,8))

## Training Score

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Spot Check Algorithms
models = []
models.append(('LR', linear_model.LinearRegression()))
models.append(('NN', MLPRegressor(solver = 'lbfgs')))  #neural network
models.append(('KNN', KNeighborsRegressor())) 
models.append(('RF', RandomForestRegressor(n_estimators = 10))) # Ensemble method - collection of many decision trees
models.append(('SVR', SVR(gamma='auto'))) # kernel = linear
# Evaluate each model in turn
results = []
names = []
for name, model in models:
    # TimeSeries Cross validation
    tscv = TimeSeriesSplit(n_splits=10)
    
    cv_results = cross_val_score(model, X, Y, cv=tscv, scoring='r2')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
# Compare Algorithms
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

In [ ]:
X[166:167]

## Linear Regression Try

In [ ]:
# example of training a final regression model
from sklearn.linear_model import LinearRegression
# fit final model
model = LinearRegression()
model.fit(X[0:160], Y[0:160])
# make a prediction
Xnew = X[160:167]
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
    j = 160
    print("X=%s, Predicted=%s, Actual=%s" % (Xnew[i], ynew[i],Y_rideType[j+i]))

In [ ]:
result = model.predict([[3,27,8,36,6,1,'9','15']])
result

## LogisticRegression for RideType

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X, Y_rideType)
result_ridetype = clf.predict([[12,27,8,36,6,1,20,3]])
print("Result type prediction=%s" % result_ridetype)

result_ridetype = clf.predict([[12,27,8,36,6,1,10,12]])
print("Result type prediction=%s" % result_ridetype)